<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-gliner-small-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/r-kovalch/acter-ner

Cloning into 'acter-ner'...
remote: Enumerating objects: 399, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 399 (delta 92), reused 91 (delta 37), pack-reused 231 (from 1)
Receiving objects: 100% (399/399), 2.53 MiB | 3.72 MiB/s, done.
Resolving deltas: 100% (246/246), done.


In [2]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 16.31 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [3]:
%cd /content/ACTER

/content/ACTER


In [4]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [5]:
ls

combine_corpora-en-fr-nl.sh  dataset_processor.py        train_full.tsv
combine_corpora-en-fr.sh     output/                     train_model.py
combine_corpora_gliner.sh    preprocess_acter_gliner.py  train_spacy_model.py
combine_corpora.sh           preprocess_acter.py         Untitled.ipynb
configs/                     test_full.tsv


In [6]:
!bash combine_corpora_gliner.sh

Writing JSON for GLiNER


In [7]:
!cp train_full.tsv train_full.jsonl && \
  cp test_full.tsv test_full.jsonl && \
  cp val_full.tsv val_full.jsonl

cp: cannot stat 'val_full.tsv': No such file or directory


In [8]:
!pip install -U "gliner>=0.2.19" "transformers>=4.51.0" \
               datasets accelerate evaluate seqeval --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s eta 0:00:00
  

In [ ]:
# 1.  Imports
from datasets import load_dataset
from gliner import GLiNER
from gliner.training import Trainer, TrainingArguments
from gliner.data_processing import GLiNERDataset, WordsSplitter
from gliner.data_processing.collator import DataCollatorWithPadding
import evaluate, torch, random, itertools, json

# 2.  Load ACTER JSONL
raw = load_dataset(
    "json",
    data_files={"train": "./train_full.jsonl",
                "validation": "./test_full.jsonl"},
)

# 3.  Convert char-level spans  ➜  GLiNER format
LABEL_SET = set()
def to_gliner(ex):
    tokens, char2tok, off = [], {}, 0
    for i, tok in enumerate(ex["text"].split()):
        tokens.append(tok)
        char2tok.update({off + j: i for j in range(len(tok))})
        off += len(tok) + 1
    ner = []
    for ent in ex["entities"]:
        s = char2tok.get(ent["start"]); e = char2tok.get(ent["end"] - 1)
        if s is not None and e is not None:
            lbl = ent["label"].lower()
            ner.append([s, e, lbl]); LABEL_SET.add(lbl)
    item = {"tokenized_text": tokens, "ner": ner}
    if not ner:                               # sentences without entities
        item["label"] = sorted(LABEL_SET)     # see issue #139
    return item

train_py = [to_gliner(x) for x in raw["train"]]
dev_py   = [to_gliner(x) for x in raw["validation"]]
labels   = sorted(LABEL_SET)

# 4.  Wrap with GLiNERDataset
model     = GLiNER.from_pretrained("gliner-community/gliner_small-v2.5")
tok       = model.data_processor.transformer_tokenizer
splitter  = WordsSplitter(model.config.words_splitter_type)

train_ds = GLiNERDataset(train_py, model.config, tok, splitter,
                         entities=labels)
dev_ds   = GLiNERDataset(dev_py,  model.config, tok, splitter,
                         entities=labels)

# 5.  Use *DataCollatorWithPadding* (NOT DataCollator)
collator = DataCollatorWithPadding(model.config)

# ---------------- 6.  compute_metrics  --------------------
seqeval = evaluate.load("seqeval")

def char_to_tokens(txt):
    """helper: char idx ➜ token idx map for whitespace split text"""
    m, p = {}, 0
    for i, t in enumerate(txt.split()):
        m.update({p + j: i for j in range(len(t))}); p += len(t) + 1
    return m

def spans_to_bio(tokens, spans, label2idx):
    tags = ["O"] * len(tokens)
    for s, e, lab in spans:
        tags[s] = f"B-{lab}"
        for i in range(s + 1, e + 1):
            tags[i] = f"I-{lab}"
    return tags

import re

# pre-compile once – matches “P: 78.42%    R: 71.95%    F1: 75.03%”
_PRF_RE = re.compile(
    r"P:\s*([0-9.]+)%\s*R:\s*([0-9.]+)%\s*F1:\s*([0-9.]+)%", re.I
)

def compute_metrics(_eval_pred):
    """
    Handles   model.evaluate() -> (output_str, f1)
    where output_str looks like:  "P: 78.42%\\tR: 71.95%\\tF1: 75.03%\\n"
    Returns ents_p / ents_r / ents_f (in %) and 'score' = F1 (0-1).
    """
    out_str, f1 = model.evaluate(                 # <- your Evaluator method
        dev_py,
        threshold=0.35,
        entity_types=labels
    )

    # -------- extract P and R from the string -----------------------------
    m = _PRF_RE.search(out_str)
    if not m:
        raise ValueError(f"Cannot parse PRF from: {out_str!r}")
    p, r = (float(m.group(1)), float(m.group(2)))   # already %
    # f1 returned by evaluate() is 0-1, convert to %
    f1_pct = f1 * 100

    return {
        "ents_p": p,
        "ents_r": r,
        "ents_f": f1_pct,
        "score":  f1,           # 0-1 scalar for best-model tracking
    }


# 6.  TrainingArguments
args = TrainingArguments(
    output_dir="gliner_acter_ft",
    learning_rate=5e-6,
    per_device_train_batch_size=8*3,
    per_device_eval_batch_size=8*3,
    num_train_epochs=35,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    report_to="none",
    seed=42,
    remove_unused_columns=False,   # keep custom keys like 'label'
)

# 7.  Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tok,                 # still accepted; FutureWarning OK
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()
model.save_pretrained("gliner_acter_en_small_v2.5_ft")


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-13-e1490785721e>:121: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch,Training Loss,Validation Loss,Ents P,Ents R,Ents F,Score
1,1.988300,1.957787,69.030000,75.820000,72.265625,0.722656
2,1.613900,1.996658,75.610000,76.230000,75.918367,0.759184


In [ ]:
# from google.colab import runtime
# runtime.unassign()